In [1]:
%load_ext autoreload
from google.auth import default
credentials, project = default()

In [2]:

%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()

from src.config_loader import config
from src.gcs_storage import read_from_gcs, upload_to_gcs,get_wiktionary_cache_path 
from src.utils import get_first_n_items
from src.wiktionary import generate_wiktionary_links
from tqdm import tqdm
config.TARGET_LANGUAGE_NAME

'Swedish'

# Wiktionary Links

```raw
These are stored as a space separated string of either the words or a <a href> <\a> item.

Within the phrase dictionary they are stored with the key 'wiktionary_link'

phrase_key : {'english' : phrase key,
            'french' : phrase key,
            'wiktionary_links' : <a href>phrase</a> key}```

We store the wiktionary links as a cache so they can be reused in, e.g. the stories

In [3]:
# get current translated JSON file
from src.gcs_storage import get_wiktionary_cache_path, get_translated_phrases_path

phrase_translations = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=get_translated_phrases_path())
#word_link_cache = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=get_wiktionary_cache_path())

In [4]:
word_link_cache= read_from_gcs(config.GCS_PRIVATE_BUCKET, file_path = get_wiktionary_cache_path())

In [5]:
from src.wiktionary import add_wiktionary_links
phrase_translations, word_link_cache = add_wiktionary_links(phrase_translations, word_link_cache, overwrite=False)

Adding wiktionary links:   0%|          | 1/838 [00:01<17:04,  1.22s/it]

modified dont_worry_the_alarm_will_sound_if_theres_danger


Adding wiktionary links:   0%|          | 2/838 [00:01<10:28,  1.33it/s]

modified lets_go_to_bed_early_so_we_feel_better_tomorrow
modified i_know_exactly_what_youre_talking_about


Adding wiktionary links:   0%|          | 4/838 [00:02<07:23,  1.88it/s]

...

modified knock_before_entering_its_common_courtesy


Adding wiktionary links: 100%|█████████▉| 836/838 [09:58<00:01,  1.77it/s]

modified lets_cover_all_aspects_of_the_problem
modified hes_trying_to_improve_his_cooking


Adding wiktionary links: 100%|██████████| 838/838 [09:59<00:00,  1.40it/s]

modified wash_your_hands_before_you_drink_tea


In [6]:
upload_to_gcs(obj=phrase_translations, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=get_translated_phrases_path(collection="LM1000"))

'gs://audio-language-trainer-private-content/collections/LM1000/swedish/translations.json'

In [7]:
upload_to_gcs(word_link_cache, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=get_wiktionary_cache_path())

'gs://audio-language-trainer-private-content/resources/swedish/swedish_wiktionary_cache.json'